#  Training an Multi-Layer Perceptron with TensorFlow's High-Level API

TensorFlow provides a simple way to create a MLP using its high-level API. The `DNNClassifier` class allows to train a neural network with any number of hidden layers and a softmax output layer. The code below creates a network with two hidden layers and a softmax output layer with 10 neurons. The code creates a set of real valued columns from the training set, creates a `DNNClassifier`, and wraps it up in a Scikit-Learn compatibility helper. Then, 40000 training iterations are ran using batches of 50 instaces. 

```python
import tensorflow as tf 

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10, 
                                        feature_columns=feature_cols)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)

from sklearn.metrics import accuracy_score
y_pred = dnn_clf.predict(X_test)
score = accuracy_score(y_test, y_pred['classes'])
```

# Training a DNN Using Plain TensorFlow

Sometimes it is necessary to have more control over the architecture of the network. In these cases, it is better to use TensorFlow's lower-level Python API. 

## Construction Phase

The first step to build a network is to specify the number of inputs, outputs, and hidden layers. Also, the placeholders that are going to hold the targets and the training data have to be created. The placeholder `X` is going to act as the input layer of the network and the values contained in this placeholder will be replaced with one training batch at a time.

In [1]:
import tensorflow as tf 

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')


The `tf.layers.dense` function creates a fully connected layer where all the inputs are connected to all the neurons in the layer. This function takes care of initializing the bias and the weights using the appropiate initialization strategy. The activation function can also be set using the `activation` parameter. The first parameter of the function is the input for the layer.

In [2]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1',
                             activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2',
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

The next step of the construction phase is defining a cost function for training; in this case the cost function is cross entropy. Cross entropy penalizes models that predict low probability for the target class. The way to compute cross entropy in TensorFlow is using the `sparse_softmax_cross_entropy_with_logits()` function. This function calculates the cross entropy before the softmax activation function (logits). This function expects that the labels are integers from 0 to the number of classes minus 1. The function outputs a vector with the cross entropy for each instance, and the function `reduce_mean()` to compute the mean cross entropy over all instances. 

In [3]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
                                                            

Once the network and the cost functions are ready, it is necessary to define an optimizer that will change the parameters to minimize the cost function. For this network, the `GradientDescentOptimizer` will be used.

In [4]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    

The last step in the construction phase is to define how the model is going to be evaluated. In this case, the accuracy measure is going to be used. The first step for checking the accuracy is determining if the prediction made by the neural network for each instance is correct. To do that, the `in_top_k()` function is used to determine if the highest logit corresponds to the target class. This function returns a tensor with booleans, so the booleans are casted to floats to compute the average. This average is the network's overall accuracy.

In [5]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## Execution Phase 

The first part of the execution phase is loading the data that is going to be used. Then, the number of epochs and the size of the mini-batches has to be defined. With the data, number of epochs, and mini-batch size ready, the model can be trained.

The code below opens a TensorFlow session and initializes all the variables using an `init` node. In each epoch, the code iterates through mini-batches that correspons to the training set size. Each batch is fetched using the `next_batch()` function. Then, the code runs the training operation with the current mini-batch input data and targets. At the end of each epoch, the code evaluates the model on the last mini_batch and on the entire test set. The parameters of the model are saved to the disk after all the epochs are ran. 

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data')

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, 'Train accuracy:', acc_train, 'Test accuracy', acc_test)
        
    save_path = saver.save(sess, './my_model_final.ckpt')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.9 Test accuracy 0.8874
1 Train accuracy: 0.94 Test accuracy 0.904
2 Train accuracy: 0.8 Test accuracy 0.9141
3 Train accuracy: 0.92 Test accuracy 0.9195
4 Train accuracy: 0.92 Test accuracy 0.9257
5 Train accuracy: 0.94 Test accuracy 0.9294
6 Train accuracy: 0.92 Test accuracy 0.9332
7 Train accuracy: 0.88 Test accuracy 0.9347
8 Train accuracy: 0.94 Test accuracy 0.9372
9 Train accuracy: 0.96 Test accuracy 0.9406
10 Train accuracy: 0.92 Test accuracy 0.9427
11 Train accuracy: 0.92 Test accuracy 0.9434
12 Train accuracy: 0.92 Te

## Using the Neural Network

Now that the network is trained and the paramaters saved in the disk, the construction phase is the same as the above but the execution phase has to be changed. The change has to load the parameters from disk and load new images that want to be classified. The new code evaluates the `logits` node, and ,if the estimated class probabilities are needed, the `softmax()` function has to be applied to the logits. However, if only the class prediction is required, it is enough to pick the class that has the highest logit value (with a function such as `argmax()`.

```python
with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    X_new_scaled = [...] #new images
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
```

# Fine-Tuning Neural Network Hyperparameters

The flexibility of neural networks also means that there are many hyperparameters no tweak. Performing a grid-search over the hyperparameter space is not feasible, because training a network on a large dataset takes a long time. Instead of the grid-search it would be better to use a randomized search or use a tool such as Oscar to find a good set of hyperparameters. Below are a few facts to have in mind regarding the hyperparameters of a network.

**Number of hidden layers**: A thumb rule is to prefer deep networks with few neurons per layer over shallow networks with many neurons per layer. Deep networks can model complex functions using exponentially less neurons than shallow nets, so they are much faster to train. 

**Number of Neurons per Hidden Layer**: It is common practice to size the layers to form a funnel. The idea behind the funnel structure is that many low-level features con converge into fewer high-level features. However, now it is also common to use the same number of neurons per layer. Also, the performance will be improved more if an extra layer is added instead of more neurons per layer. 

**Activation Functions**: The ReLU function is often used in hidden layers, because it is faster to compute than other activation functions and because Gradient Descent does not get stuck as much on plateus. The latter is due to the fact that the ReLU function does not saturate for large input values. In the output layer, it is common to use a softmax activation function for classes that are mutually exclusive. For regression tasks, an activation in this layer is not required.